Import relevant packages

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import json
import numpy as np
import random
import os

#tf.enable_eager_execution()
#tf.compat.v1.set_random_seed(18)

Set up data loading functions.

In [3]:
tf.__version__

'2.1.0'

In [4]:
# One shot learning - Each network takes input and output and tries to compare if two tasks are the same.
Xt = []
Yt = []
Xv = []
Yv = []

rng = np.random.default_rng()

def load(name):
    with open(name) as json_file:
        data = json.load(json_file) 
        ctraintask = []
        cvaltask = []
        l = len(data['train'])
        i = 0
        for t in data['train']:
            i += 1
            cin = np.array(t['input'])
            cinr = np.full((30, 30), -1)
            cinr[:cin.shape[0], :cin.shape[1]] = cin
            cout = np.array(t['output'])
            coutr = np.full((30, 30), -1)
            coutr[:cout.shape[0], :cout.shape[1]] = cout
            cX = np.concatenate((cinr, coutr))
            if not i == l:
                ctraintask.append(cX)
                Yt.append(name)
            else:
                cvaltask.append(cX)
                Yv.append(name)
        Xt.append(np.array(ctraintask))
        Xv.append(np.array(cvaltask))

def testAccuracy(model, N, loops):
    right = 0
    wrong = 0
    for _ in range(loops):
        basecategory = random.randint(0, Xv.shape[0]-1)
        baseTask = Xv[basecategory][0].reshape(60, 30, 1)
        baseProb = model.predict([np.array([baseTask]), np.array([Xt[basecategory][random.randint(0, Xt[basecategory].shape[0]-1)].reshape(60, 30, 1)])])[0][0]
        for i in range(0, N):
            ccategory = (basecategory+random.randint(1, Xt.shape[0]))%Xt.shape[0]
            cindex = random.randint(0, Xt[ccategory].shape[0]-1)
            ctask = Xt[ccategory][cindex] 
            cProb = model.predict([np.array([baseTask]), np.array([ctask.reshape(60, 30, 1)])])
            if cProb[0][0] > baseProb:
                wrong += 1
                break
        else:
            right += 1
    return (100*right)/loops

def getBatch(batchsize, training):
    if training:
        X = Xt
        Y = Yt
    else:
        X = Xv
        Y = Yv
    categories = rng.choice(X.shape[0], size=(batchsize,), replace=False)
    targets = np.zeros((batchsize,))
    targets[batchsize//2:] = 1
    pairs=[np.zeros((batchsize, 60, 30, 1)) for i in range(2)]
    for i in range(batchsize):
        category = categories[i]
        pairs[0][i,:,:,:] = X[category][random.randint(0, X[category].shape[0]-1)].reshape(60, 30, 1)
        if i >= batchsize // 2:
            category_2 = category
        else:
            category_2 = (category + random.randint(1, X.shape[0])) % X.shape[0] 
        pairs[1][i,:,:,:] = X[category_2][random.randint(0, X[category_2].shape[0]-1)].reshape(60, 30, 1)
    return pairs, targets

def generate(batchsize, training):
    while True:
        pairs, targets = getBatch(batchsize, training)
        yield (pairs, targets)

Load data.

In [5]:
with os.scandir('training/') as entries:
    for entry in entries:
        load("training/"+entry.name)

with os.scandir('test/') as entries:
    for entry in entries:
        load("test/"+entry.name)

Xt = np.array(Xt)
Xv = np.array(Xv)
Yt = np.array(Yt)
Yv = np.array(Yv)

Define model.

In [6]:
# This function is based off the example in this tutorial: https://towardsdatascience.com/one-shot-learning-with-siamese-networks-using-keras-17f34e75bb3d
def get_siamese_model(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    model = Sequential()
    model.add(Conv2D(64, (10,10), padding="same", activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), padding="same", activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), padding="same", activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), padding="same", activation='relu'))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid', name='taskrepresentation'))

    encoded_l = model(left_input)
    encoded_r = model(right_input)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

    return siamese_net

In [7]:
model = get_siamese_model((60, 30, 1))

In [8]:
model.get_layer("sequential_3").outputs[0]

ValueError: No such layer: sequential_3

Train model.

In [9]:
model = get_siamese_model((60, 30, 1))
print(model.summary())
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy", optimizer=optimizer)
print(testAccuracy(model, 10, 18))

def trainmodel(epo):
    bestAccuracy = 0

    for i in range(epo):
        model.fit_generator(generate(128, True), epochs=1, steps_per_epoch=50)
        accuracy = testAccuracy(model, 20, 18)
        if accuracy > bestAccuracy:
            model.save_weights("bestCheckpoint/best")
            bestAccuracy = accuracy
            print("New Best: "+str(bestAccuracy))

trainmodel(100)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 60, 30, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 60, 30, 1)]  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         23219008    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           sequential_1[1][0]         

KeyboardInterrupt: 

In [ ]:
n = 12
k = 10
ctask1 = Xt[n][0].reshape(60, 30, 1)
plt.imshow(ctask1.reshape(60, 30))
plt.colorbar()
plt.show()
ctask2 = Xv[k][0].reshape(60, 30, 1)
plt.imshow(ctask2.reshape(60, 30))
plt.colorbar()
plt.show()
taskin = []
taskin.append(tf.convert_to_tensor(np.array([ctask1.astype(np.float32)])))
taskin.append(tf.convert_to_tensor(np.array([ctask2.astype(np.float32)])))

print(model(taskin))

Adversarial Generation.

In [ ]:
import matplotlib.pyplot as plt
model = get_siamese_model((60, 30, 1))
model.load_weights("bestCheckpoint/best")

In [ ]:
from timeit import default_timer as timer
import random

In [ ]:
#sampletask = [np.array([AI.Xt[0][0].reshape(60,30,1)]), np.array([AI.Xt[1][1].reshape(60,30,1)])]
loss_object = tf.keras.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

def getGradients(taskin):
    taskin[0] = tf.convert_to_tensor(taskin[0].astype(np.float32))
    taskin[1] = tf.convert_to_tensor(taskin[1].astype(np.float32))
    with tf.GradientTape() as tape:
        tape.watch(taskin)
        prediction = model(taskin)
        loss = loss_object(tf.convert_to_tensor(np.array([1]).reshape(1, 1)), prediction)
    gradient = tape.gradient(loss, taskin)
    #signs = tf.sign(gradient)
    return [gradient[1].numpy()[0][30:], loss, prediction]

def generateTask(examples, cin, l, accthresh, verbose):
    cin = cin.astype(np.float64)
    lasterr = 99999
    for i in range(1000):
        grads = np.zeros((30, 30, 1))
        err = 0
        acc = 0
        for e in examples:
            sampletask = [np.array([e]), np.array([cin.reshape(60,30,1)])]
            result = getGradients(sampletask)
            grads += result[0]
            err += result[1].numpy()
            acc += result[2].numpy()[0][0]
        grads /= len(examples)
        err /= len(examples)
        acc /= len(examples)
        grads *= l
        adjustment = np.zeros((60,30,1))
        adjustment[30:, :, :] = grads
        cin -= adjustment
        np.clip(cin, -1, 9, out=cin)
        if verbose:
            if i%100 == 0:
                print("EPOCH "+str(i)+": "+str(err)+" "+str(acc))
            if i%500 == 0:
                out = cin.reshape(60, 30)[30:, :]
                plt.imshow(out)
                plt.colorbar()
                plt.show()
        if acc*100 > accthresh:
            if verbose:
                print("STOPPING AT "+str(err))
            break
    out = cin.reshape(60, 30)[30:, :]
    return out

def generateTaskAdam(examples, cin, l, accthresh, verbose):
    cutoff = 10
    referencetasks = []
    outputtasks = []
    cin = cin.reshape(60, 30, 1)
    for e in examples:
        referencetasks.append(e)
        outputtasks.append(cin)
    referencetasks = tf.Variable(tf.convert_to_tensor(np.array(referencetasks).astype(np.float32)))
    adjusttask = tf.Variable(tf.convert_to_tensor(np.array(cin[30:, :]).astype(np.float32)))  
    outputtasks = tf.Variable(tf.convert_to_tensor(np.array(outputtasks).astype(np.float32)))
    starttime = timer()
    for i in range(1000):
        if timer()-starttime > 5:
            if verbose:
                print("MAX TIME EXCEEDED")
            break
        taskin = [referencetasks, outputtasks]
        with tf.GradientTape() as tape:
            tape.watch(taskin)
            prediction = model(taskin)
            loss = loss_object(tf.convert_to_tensor(np.array([1]*len(examples)).reshape(prediction.shape)), prediction)
        gradient = tape.gradient(loss, taskin)
        grad = np.mean(gradient[1], axis=0)[30:, :]
        opt.apply_gradients(zip([grad], [adjusttask]))
        
        outputtasks = []
        cin[30:, :] = adjusttask.numpy()
        for e in examples:
            outputtasks.append(cin)
        outputtasks = tf.Variable(tf.convert_to_tensor(np.array(outputtasks).astype(np.float32)))
        acc = np.mean(prediction)*100
        if acc > accthresh:
            if verbose:
                print("ENDING AT EPOCH "+str(i)+": "+str(np.mean(loss))+" "+str(acc))
            break
        if i%100 == 0:
            if verbose:
                print("EPOCH "+str(i)+": "+str(np.mean(loss))+" "+str(acc))
    out = cin.reshape(60, 30)[30:, :]
    return out

def generateExample(taskclass, thresh, adam):
    examples = []
    for i in range(Xt[taskclass].shape[0]):
        examples.append(Xt[taskclass][i].reshape(60,30,1).astype(np.float64))
    cin = np.array([Xv[taskclass][0].reshape(60,30,1)])
    cin[0][30:, :, :] = cin[0][:30, :, :]
    if adam:
        out = generateTaskAdam(examples, cin, 0.5, thresh, False)
    else:
        out = generateTask(examples, cin, 0.5, thresh, False)
    cout = cin[0].reshape(60, 30)
    cout[30:, :] = out
    return cout

Adversarial Loop

In [ ]:
Xadv = []
Yadv = []
def genAdv(batchsize):
    X = Xt
    Y = Yt
    categories = rng.choice(X.shape[0], size=(batchsize,), replace=False)
    targets = np.zeros((batchsize,))
    pairs = [np.zeros((batchsize, 60, 30, 1)) for i in range(2)]
    for i in range(batchsize):
        category = categories[i]
        pairs[0][i, :, :, :] = X[category][random.randint(0, X[category].shape[0]-1)].reshape(60, 30, 1)
        pairs[1][i, :, :, :] = generateExample(category, 80, True).reshape(60, 30, 1)
    return pairs, targets

def getBatchMixed(batchsize, training):
    pairs1, targets1 = getBatch(batchsize//2, training)
    pairs2, targets2 = genAdv(batchsize//2)
    pairs = []
    targets = np.append(targets1, targets2)
    pairs.append(np.append(pairs1[0], pairs2[0], axis=0))
    pairs.append(np.append(pairs1[1], pairs2[1], axis=0))
    return pairs, targets

def generateMixed(batchsize, training):
    while True:
        pairs, targets = getBatchMixed(batchsize, training)
        yield (pairs, targets)

optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [ ]:
trackedclasses = rng.choice(Xt.shape[0], size=(5,), replace=False)

print("Ground Truth")
for i in trackedclasses:
    correct = Xv[i][0]
    plt.imshow(correct)
    plt.colorbar()
    plt.show()

epo = 100
for i in range(epo):
    model.fit_generator(generateMixed(128, True), epochs=1, steps_per_epoch=1)
    accuracy = testAccuracy(model, 20, 18)
    print("EPOCH "+str(i)+" ACCURACY "+str(accuracy))
    for k in trackedclasses:
        out = generateExample(k, 80, True)
        plt.imshow(out)
        plt.colorbar()
        plt.show()

In [ ]:
taskclass = 5
examples = []
for i in range(Xt[taskclass].shape[0]):
    examples.append(Xt[taskclass][i].reshape(60,30,1).astype(np.float64))
cin = np.array([Xv[taskclass][0].reshape(60,30,1)])
cin[0][30:, :, :] = cin[0][:30, :, :]
generateTaskAdam(examples, cin, 0.5, 80, False)

*OPTIONAL* Test to compare ADAM to Batch Gradient Descent

In [ ]:
cclass = random.randint(0, Xt.shape[0])
t1 = timer()
out = generateExample(cclass, 80, True)
t2 = timer()
outtrad = generateExample(cclass, 80, False)
t3 = timer()
correct = Xv[cclass][0]
print("ADAM took "+str(t2-t1))
plt.imshow(out)
plt.colorbar()
plt.show()
print("Batch Gradient Descent took "+str(t3-t2))
plt.imshow(outtrad)
plt.colorbar()
plt.show()
print("Ground truth")
plt.imshow(correct)
plt.colorbar()
plt.show()

*OPTIONAL* Test to calculate average time

In [ ]:
cclass = random.randint(0, Xt.shape[0])
ttimes = 0
cycles = 40
for i in range(cycles):
    cclass = random.randint(0, Xt.shape[0]-1)
    t1 = timer()
    out = generateExample(cclass, 80, True)
    t2 = timer()
    ttimes += t2-t1
    print(t2-t1)
avgtime = ttimes/cycles
print("AVERAGE TIME "+str(avgtime))

In [ ]:
Xadv = []
Yadv = []
for i in range(Xt.shape[0]):

In [ ]:
print(Xt.shape[0])
n = 170
print(len(Xt[n]))
plt.imshow(Xt[n][1].reshape(60, 30))
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(Xt[n][0].reshape(60, 30))
plt.colorbar()
plt.show()
plt.imshow(Xt[n][1].reshape(60, 30))
plt.colorbar()
plt.show()
plt.imshow(Xv[n][0].reshape(60, 30))
plt.colorbar()
plt.show()
examples = [Xt[n][0].reshape(60,30,1).astype(np.float64), Xt[n][1].reshape(60,30,1).astype(np.float64)] 
cin = np.array([Xv[n][0].reshape(60,30,1)])
cin[0][30:, :, :] = cin[0][:30, :, :]
#cin[0][30:, :, :] = np.random.randint(-1, 10, size=(30, 30, 1))
#cin[0][30:, :, :] = np.zeros((30, 30, 1))
#sampletask[1][0][30:, :, :] = np.random.randint(-1, 11, size=(30, 30, 1))
#plt.imshow(cin.reshape(60, 30))
#plt.colorbar()
#plt.show()
#out = generateTask(examples, cin, 10, 40000)
#plt.imshow(out)
#plt.colorbar()
#plt.show()